In [ ]:
from scipy.sparse import csr_matrix
from scipy.sparse import save_npz
from implicit.als import AlternatingLeastSquares
from scipy.sparse import load_npz


/home/slahlou/anaconda3/envs/genAI/lib/python3.12/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/home/conda/feedstock_root/build_artifacts/implicit_1724419594932/work/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [ ]:
interaction_matrix = load_npz("dataset/collaborative-filtering-data/interaction_matrix.npz")


In [3]:
model = AlternatingLeastSquares(factors=64, regularization=0.05, alpha=2.0)
model.fit(interaction_matrix)

/home/slahlou/anaconda3/envs/genAI/lib/python3.12/site-packages/implicit/cpu/als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()


  0%|          | 0/15 [00:00<?, ?it/s]

In [43]:
with open("dataset/collaborative-filtering-data/user_id_map.json") as f:
    user_id_map = {int(k): v for k, v in json.load(f).items()}

with open("dataset/collaborative-filtering-data/item_id_map.json") as f:
    item_id_map = {int(k): v for k, v in json.load(f).items()}

In [ ]:
userid = 12345
userid = [k for k, v in user_id_map.items() if v == '84eba3a874e7b5660ee2803fe64ce82954a2beb56e39ed9a561e01889b9b82b8'][0]
print(userid) 
ids, scores = model.recommend(userid, interaction_matrix[userid], N=10, filter_already_liked_items=False)

707141


In [45]:
print('ids:', ids)
print('scores:', scores)

ids: [26937 42514 60079 20585 47250  1711 45748 78573 66364 51608]
scores: [0.16587898 0.12054601 0.10726665 0.10411339 0.10074175 0.09985609
 0.09893231 0.09795658 0.09420587 0.08490369]


In [46]:
already_liked = np.in1d(ids, interaction_matrix[userid].indices)
article_ids = [item_id_map[i] for i in ids]

result = pd.DataFrame({"article_id": article_ids, "score": scores, "already_bought": already_liked})

In [47]:
articles_df = pd.read_csv('dataset/datas/articles.csv')
customer_df = pd.read_csv('dataset/datas/customers.csv')

In [48]:
result

,article_id,score,already_bought
0,618800001,0.165879,True
1,673396002,0.120546,False
2,727029001,0.107267,False
3,589599047,0.104113,False
4,688537012,0.100742,False
5,372860001,0.099856,False
6,684209014,0.098932,False
7,795440001,0.097957,False
8,747939002,0.094206,False
9,699423003,0.084904,True


In [61]:
merge_result = pd.merge(result, articles_df.drop(columns=['product_code', 'perceived_colour_master_id', 'colour_group_code', 'department_no', 'product_type_no', 'graphical_appearance_no', 'index_code', 'index_group_no', 'section_no', 'garment_group_no'])
                        , on='article_id', how='left')

In [62]:
merge_result

,article_id,score,already_bought,prod_name,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_name,department_name,index_name,index_group_name,section_name,garment_group_name,detail_desc
0,618800001,0.165879,True,Leona,Vest top,Garment Upper body,Solid,Black,4,Dark,Black,Blouse,Ladieswear,Ladieswear,Womens Everyday Collection,Blouses,V-neck satin top with a lace trim at the top a...
1,673396002,0.120546,False,Ringo hipbelt,Belt,Accessories,Solid,Black,4,Dark,Black,Belts,Ladies Accessories,Ladieswear,Womens Big accessories,Accessories,Belt in grained imitation leather with a metal...
2,727029001,0.107267,False,S Timeless High Wide Side Brie,Swimwear bottom,Swimwear,All over pattern,Black,4,Dark,Black,Swimwear,Lingeries/Tights,Ladieswear,"Womens Swimwear, beachwear",Swimwear,"Fully lined, patterned bikini bottoms with a h..."
3,589599047,0.104113,False,Shake it in Balconette,Bikini top,Swimwear,All over pattern,Black,4,Dark,Black,Swimwear,Lingeries/Tights,Ladieswear,"Womens Swimwear, beachwear",Swimwear,"Lined balconette bikini top with underwired, l..."
4,688537012,0.100742,False,Simple as that Cheeky Tanga,Swimwear bottom,Swimwear,All over pattern,Black,4,Dark,Black,Swimwear,Lingeries/Tights,Ladieswear,"Womens Swimwear, beachwear",Swimwear,Fully lined bikini bottoms with a mid waist an...
5,372860001,0.099856,False,7p Basic Shaftless,Socks,Socks & Tights,Solid,Black,4,Dark,Black,Shopbasket Socks,Lingeries/Tights,Ladieswear,"Womens Nightwear, Socks & Tigh",Socks and Tights,Fine-knit trainer socks in a soft cotton blend.
6,684209014,0.098932,False,Simple as That Triangle Top,Bikini top,Swimwear,All over pattern,Black,4,Dark,Black,Swimwear,Lingeries/Tights,Ladieswear,"Womens Swimwear, beachwear",Swimwear,"Lined, non-wired, triangle bikini top with a w..."
7,795440001,0.097957,False,Juan,Vest top,Garment Upper body,Solid,Black,4,Dark,Black,Jersey fancy,Ladieswear,Ladieswear,Womens Everyday Collection,Jersey Fancy,"Top in softly draping, ribbed jersey with a sh..."
8,747939002,0.094206,False,Love Shack Cheeky Brief,Swimwear bottom,Swimwear,All over pattern,Off White,3,Light,White,Swimwear,Lingeries/Tights,Ladieswear,"Womens Swimwear, beachwear",Swimwear,"Fully lined, patterned bikini bottoms with a m..."
9,699423003,0.084904,True,Ring Ring Triangle Top,Bikini top,Swimwear,All over pattern,Light Orange,1,Dusty Light,Orange,Swimwear,Lingeries/Tights,Ladieswear,"Womens Swimwear, beachwear",Swimwear,Lined triangle bikini top with removable inser...


In [51]:
real_customer_id = user_id_map[userid]
real_customer_id

'84eba3a874e7b5660ee2803fe64ce82954a2beb56e39ed9a561e01889b9b82b8'

In [52]:
customer_df[customer_df['customer_id'] == real_customer_id]

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
712121,84eba3a874e7b5660ee2803fe64ce82954a2beb56e39ed...,1.0,1.0,ACTIVE,Regularly,24.0,aa0a1031658179805fd3d4a5445ba831835a1c3f8ed097...


In [53]:
transaction_df = pd.read_csv('dataset/datas/transactions_train.csv')

In [54]:
user_purshases = transaction_df[transaction_df['customer_id'] == real_customer_id]

In [55]:
user_purshases = pd.merge(user_purshases, 
							articles_df.drop(columns=['product_code', 'product_type_no', 'graphical_appearance_no', 'index_code', 'index_group_no', 'section_no', 'garment_group_no']),
							on='article_id', how='left'
						  )

In [60]:
user_purshases[['article_id', 'prod_name', 'product_type_name', 'product_group_name', 'graphical_appearance_name', 'section_name', 'garment_group_name', 'detail_desc']]

,article_id,prod_name,product_type_name,product_group_name,graphical_appearance_name,section_name,garment_group_name,detail_desc
0,677219002,Larry basic sweater,Sweater,Garment Upper body,Solid,Womens Casual,Knitwear,Wide jumper in a soft fine knit containing som...
1,557247018,Petar Sweater,Sweater,Garment Upper body,Solid,Men Underwear,Jersey Basic,Oversized top in sturdy sweatshirt fabric with...
2,501820003,SIRPA,Sweater,Garment Upper body,Melange,Divided Collection,Knitwear,Jumper in a soft knit with a slightly wider ne...
3,610744009,Macy,Sweater,Garment Upper body,Solid,Womens Everyday Basics,Knitwear,Jumper in a soft knit with low dropped shoulde...
4,610744007,Macy,Sweater,Garment Upper body,Solid,Womens Everyday Basics,Knitwear,Jumper in a soft knit with low dropped shoulde...
5,775996009,Alex sweater,Sweater,Garment Upper body,Dot,Womens Casual,Knitwear,Boxy-style jumper in a soft knit containing so...
6,767422003,New York Jaquard,Sweater,Garment Upper body,Jacquard,Divided Collection,Knitwear,Boxy-style jumper in a jacquard knit with a so...
7,805323002,Bow PU trouser,Trousers,Garment Lower body,Solid,Womens Everyday Collection,Trousers,"High-waisted, ankle-length trousers in imitati..."
8,618800001,Leona,Vest top,Garment Upper body,Solid,Womens Everyday Collection,Blouses,V-neck satin top with a lace trim at the top a...
9,842678001,Romana,Sweater,Garment Upper body,Solid,Womens Everyday Collection,Knitwear,"Jumper in a soft knit with a ribbed polo neck,..."
